# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### ABRE NAVEGADOR ###
navegador = webdriver.Chrome() #abre um navegador chrome e guarda o objeto de interação com o navegador

### BUSCA AS COTAÇÕES NA WEB ###

#DOLAR
navegador.get("https://www.google.com/") #acessa um site

#encontra um elemento pelo seu xpath e envia um texto para este elemento
navegador.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação dolar") #escreve no campo
navegador.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(Keys.ENTER) #aperta enter

cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") #retorna o valor daquele atributo
print(cotacao_dolar)

#EURO
navegador.get("https://www.google.com/") #acessa um site

#encontra um elemento pelo seu xpath e envia um texto para este elemento
navegador.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação euro") #escreve no campo
navegador.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(Keys.ENTER) #aperta enter

cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") #retorna o valor daquele atributo
print(cotacao_euro)

#OURO
navegador.get("https://www.melhorcambio.com/ouro-hoje") #acessa um site

cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(',','.') #substitui as virgulas por pontos (válido pois é uma variável string)
print(cotacao_ouro)

navegador.quit()

5.2199
6.127755447
294.12


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)


tabela["Preço Base Reais"] = tabela["Preço Base Original"]*tabela["Cotação"]
tabela["Preço Final"] =  tabela["Preço Base Reais"]*tabela["Margem"]
display(tabela)

tabela.to_excel("Preços_Atualizado.xlsx", index = False)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.219900,5219.847801,1.40,7307.786921
1,Carro Renault,4500.00,Euro,6.127755,27574.899511,2.00,55149.799023
2,Notebook Dell,899.99,Dólar,5.219900,4697.857801,1.70,7986.358262
3,IPhone,799.00,Dólar,5.219900,4170.700100,1.70,7090.190170
4,Carro Fiat,3000.00,Euro,6.127755,18383.266341,1.90,34928.206048
5,Celular Xiaomi,480.48,Dólar,5.219900,2508.057552,2.00,5016.115104
6,Joia 20g,20.00,Ouro,294.120000,5882.400000,1.15,6764.760000
